In [131]:
from glob import glob
import pandas as pd
import numpy as np
import re
import time
from translate import Translator


In [152]:
translator= Translator(to_lang="en",from_lang="hy")
#putting all 5 dfs into one list and dropping rows where all values are NA
allFiles =glob("oct*.csv")
ls=[]
frame = pd.DataFrame()

In [161]:
for i in allFiles:
    ls.append(pd.read_csv(i,index_col=None))

In [162]:
sep=pd.concat(ls)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [163]:
sep=sep.loc[:, sep.columns != 'Unnamed: 0']

In [164]:
sep.columns

Index(['brand', 'date', 'id_val', 'model', 'price', 'year', 'Անվահեծերը',
       'Գույնը', 'Դռների քանակը', 'Թափքը', 'Ձիաուժը', 'Ղեկը',
       'Մխոցների քանակը', 'Մոդիֆիկացիան', 'Շարժիչը', 'Շարժիչի ծավալը',
       'Սրահի գույնը', 'Վազքը', 'Փոխանցման տուփը', 'Քարշակը'],
      dtype='object')

In [165]:
new_columns=['brand', 'date', 'id_val', 'model', 'price', 'year', 'Wheels',
       'Color', 'Door_Count', 'Body_Style', 'Horsepower', 'Hand_Drive',
       'Engine_Cylinders', 'Version', 'Engine', 'Engine_Volume',
       'Interior_Color', 'Mileage', 'Gearbox', 'Drive_train']

In [166]:
sep.columns=new_columns

In [167]:
sep["date"]=sep.date.apply(lambda x:re.findall("[\d\.]+",x)[0])
sep['date']=pd.to_datetime(sep['date'])

In [168]:
Body_Styles=sep.Body_Style.value_counts().index
colors=sep.Color.value_counts().index
Hand_Drive=sep.Hand_Drive.value_counts().index
engine=sep.Engine.value_counts().index
inter=sep.Interior_Color.value_counts().index
Gearbox=sep.Gearbox.value_counts().index.tolist()
Drive=sep.Drive_train.value_counts().index

In [169]:
dct_body={k:translator.translate(k) for k in Body_Styles}
dct_col={k:translator.translate(k) for k in colors}
dct_hand={k:translator.translate(k) for k in Hand_Drive}
dct_eng={k:translator.translate(k) for k in engine}
dct_inter={k:translator.translate(k) for k in inter}
dct_Gearbox={k:translator.translate(k) for k in Gearbox}
dct_Drive={k:translator.translate(k) for k in Drive}

In [170]:
sep.replace({"Body_Style": dct_body,"Color":dct_col,"Hand_Drive":dct_hand,"Engine":dct_eng,"Interior_Color":dct_inter,
            "Gearbox":dct_Gearbox,"Drive_train":dct_Drive},inplace=True)

In [171]:
def clean_price(x):
    if x.endswith("$"):
        x=float(re.findall('[\d]+',x)[0]+re.findall('[\d]+',x)[1])
    elif x.endswith("֏"):
        x=float(re.findall('[\d]+',x)[0]+re.findall('[\d]+',x)[1])/485
    else:
        x=float(re.findall('[\d]+',x)[0]+re.findall('[\d]+',x)[1])*1.13
    return x

In [172]:
sep['price']=sep['price'].apply(clean_price)
sep.Wheels=sep.Wheels.apply(lambda x:re.findall("[\d]+",x)[0] if pd.notnull(x) else x)
sep.drop(columns="Door_Count",inplace=True)
sep.Horsepower=sep.Horsepower.apply(lambda x:float(x) if str(x).isdigit() else x)
sep.Mileage=sep.Mileage.apply(lambda x: float(re.findall("[\d]+",x)[0]))

In [173]:
sep.to_csv("October.csv",index=False)
sep=pd.read_csv("October.csv", parse_dates=['date'])
sep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36714 entries, 0 to 36713
Data columns (total 19 columns):
brand               36714 non-null object
date                36714 non-null datetime64[ns]
id_val              36714 non-null object
model               36714 non-null object
price               36714 non-null float64
year                36714 non-null int64
Wheels              9508 non-null float64
Color               36714 non-null object
Body_Style          36714 non-null object
Horsepower          8890 non-null object
Hand_Drive          36714 non-null object
Engine_Cylinders    8634 non-null float64
Version             18996 non-null object
Engine              36714 non-null object
Engine_Volume       11932 non-null float64
Interior_Color      10680 non-null object
Mileage             36714 non-null float64
Gearbox             36714 non-null object
Drive_train         10124 non-null object
dtypes: datetime64[ns](1), float64(5), int64(1), object(12)
memory usage: 5.3+ MB
